# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [122]:
def argsort(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)

def formula(a,b,c):
    if c == 0:
        return 0
    else:
        return a-((b**2)/c)

def mean(y):
    if len(y) == 0:
        return 0
    else:
        return sum(y)/len(y)
    
def mean_from_sum(sum_res, length):
    if length == 0:
        return 0
    else:
        return sum_res/length

def get_threshold(feature, y, train_mode =0 ):
    indices = argsort(feature)
    
    best_thr = feature[indices[0]]-1
    
    left_sum = 0
    left_sum_sq = 0
    left_num = 0
    left_mean = 0
    
    right_sum = sum(y)
    right_sum_sq = sum(y**2)
    right_num = len(y)
    right_mean = mean_from_sum(right_sum, right_num)
    
    right_res = formula(right_sum_sq, right_sum, right_num)
    left_res = formula(left_sum_sq, left_sum, left_num)
    
    l_mean = left_mean
    r_mean = right_mean
    
    f_best = right_res + left_res
    f_score = f_best
    
    for i in indices:
        left_sum += y[i]
        left_sum_sq += y[i]**2
        left_num += 1
        left_mean = mean_from_sum(left_sum, left_num)
        
        right_sum -= y[i]
        right_sum_sq -= y[i]**2
        right_num -= 1
        right_mean = mean_from_sum(right_sum, right_num)
        
        right_res = formula(right_sum_sq, right_sum, right_num)
        left_res = formula(left_sum_sq, left_sum, left_num)

        f_score = right_res + left_res
        if f_score < f_best:
            f_best = f_score
            best_thr = feature[i]
            l_mean = left_mean
            r_mean = right_mean
    return f_best, best_thr, l_mean, r_mean


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None
    for j in range(X.shape[1]):
        f_score, f_thr, l_mean, r_mean = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
            left_mean = l_mean
            right_mean = r_mean
    return best_f, best_thr, left_mean, right_mean


def predict(result, X):
    i = result[0]
    thr = result[1]
    left_res = result[2]
    right_res = result[3]
    y = []
    for row in X:
        if row[i] > thr:
            y.append(right_res)
        else:
            y.append(left_res)
    return y
    

In [123]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [124]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848


In [125]:
result = train_stump(X,y)
result

(5, 6.939, 19.933720930232564, 37.23815789473709)

In [126]:
predictions = predict(result , X)

In [127]:
my_error = mean_squared_error(y, predictions)
error = mean_squared_error(y, regressor_stump.predict(X))
print(f"my error = {my_error}")
print(f"sk_learn error = {error}")
print(f"error diff = {my_error - error}")

my error = 46.19909167710848
sk_learn error = 46.19909167710848
error diff = 0.0
